# Figure 10

## Imports

In [ ]:
import dolfin  
import matplotlib.pyplot as plt
import numpy   
import sympy   
import sys
import os

import dolfin_mech     as dmech
import micro_poro_structure_generator as gen


## Defining geometry and material parameters

In [ ]:
seeds_filename = "Fig10-seeds.dat"
mesh_filebasename = "Fig10-mesh"

domain_y = 1.
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.092

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = 0.,
    remove_seeds = True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)


dV = dolfin.Measure("dx",domain=mesh)
coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])

vol = (xmax - xmin)*(ymax - ymin)
bbox = [xmin, xmax, ymin, ymax]

In [ ]:
Es     = 1.  # (Pa)
nus    = 0.499 # ()

mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

### Homogenized problem

In [ ]:
homogenization_problem = dmech.HomogenizationProblem(
        dim=2,
        mesh=mesh,
        mat_params=mat_params["parameters"],
        vol=vol,
        bbox=bbox)
mu_tilde, lmbda_tilde = homogenization_problem.get_lambda_and_mu()

lmbda = sympy.Symbol('lambda') # lambda is a reserved keyword in python, it cannot be used as a name for a variable.
mu    = sympy.Symbol('mu')

subs_dict        = {}
subs_dict[lmbda] = lmbda_tilde
subs_dict[mu]    = mu_tilde

## Computing macroscopic model response

### Kinematics variables

In [ ]:
# Cauchy-Green dilatation tensor
C_var = sympy.MatrixSymbol('C', 2, 2).as_explicit()
print ("C_var:"); C_var

C_inv_var = sympy.Inverse(C_var)
print ("C_inv_var:"); C_inv_var.doit()

# Invariants
I_C_var   = sympy.trace(C_var)
III_C_var = sympy.det(C_var)
J_var     = sympy.sqrt(III_C_var)
print (  "I_C:"); I_C_var
print ("III_C:"); III_C_var
print (    "J:"); J_var

## Computing macroscopic model response

### Deviatoric deformation

In [ ]:
# Deformation gradient
# In principle we could define X and x and use tensor derivative, but that is a bit involved.
beta = sympy.symbols('beta')
F = sympy.Matrix(
    [[beta, 0     ],\
     [0   , 1/beta]])
print ("F:"); F

# Determinant of deformation gradient (a.k.a. Jacobian)
J = F.det()
print ("J:"); J

# Right Cauchy-Green dilatation tensor
C = F.T * F
print ("C:"); C

C_inv = sympy.Inverse(C).doit()
print ("C_inv:"); C_inv

#### Coupled model

In [ ]:
p_f = sympy.symbols('p_f')

# Neo-Hookean free energy potential
W_c =   (lmbda/4) * (J_var**2 - 1 - 2 * sympy.ln(J_var)) \
      + (   mu/2) * (I_C_var  - 2 - 2 * sympy.ln(J_var))
# print ("W_c:"); W_c
print ("W_c:"); W_c.simplify()

# Second Piola-Kirchhoff stress tensor
Sigma_c = 2*sympy.diff(W_c, C_var) - p_f * J_var * C_var.inv()
# print ("Sigma_c:"); Sigma_c
# print ("Sigma_c:"); Sigma_c.doit()
# print ("Sigma_c:"); Sigma_c.doit().as_explicit()

p_c = - (sympy.trace(Sigma_c.T * C_var) + lmbda/2 * (J_var**2 - 1))/3/J_var
# print ("p_c:"); p_c

p_c_shear = p_c.subs(list(zip(C_var, C))).doit()
print ("p:"); p_c_shear.simplify()

#### Decoupled model

In [ ]:
K = lmbda + 2*mu/3
G = mu

# Neo-Hookean free energy potential
W_d = (K/4) * (J_var**2 - 1 - 2 * sympy.ln(J_var)) \
    + (G/2) * (J_var**(-2/3)*(1+ I_C_var) - 3)
print ("W_d:"); W_d.simplify()  

# Second Piola-Kirchhoff stress tensor
Sigma_d = 2*sympy.diff(W_d, C_var) - p_f * J_var * C_var.inv()
# print ("Sigma_d:"); Sigma_d
# print ("Sigma_d:"); Sigma_d.doit()
# print ("Sigma_d:"); Sigma_d.doit().as_explicit()

p_d = - (sympy.trace(Sigma_d.T * C_var) + J_var**(-2/3)*(1 - (sympy.Trace(C_var) + 1)/3)*G + K/2 * (J_var**2 - 1))/3/J_var
# print ("p_d:"); p_d

p_d_shear = p_d.subs(list(zip(C_var, C))).doit()
print ("p:"); p_d_shear.simplify()

In [ ]:
lam_c = sympy.lambdify(beta, p_c_shear.subs(subs_dict).subs(p_f, 0), modules=['numpy'])
lam_d = sympy.lambdify(beta, p_d_shear.subs(subs_dict).subs(p_f, 0), modules=['numpy'])
beta_vals = numpy.linspace(0, 2, 100)
p_lst_macro_c = lam_c(beta_vals)
p_lst_macro_d = lam_d(beta_vals)

## Computing microscopic model response

### Coupled model

In [ ]:
qois_filename = "Fig10-qois.dat"
res_basename = "Fig10"

mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

load_params = {}

pf = 0
beta_lst = [x/10 for x in range(10, 21)]
load_params["pf_lst"] = [0 for pf_ in range(len(beta_lst))]
load_params["sigma_bar_01_lst"] = [0 for sigma_ in range(len(beta_lst))]
load_params["sigma_bar_10_lst"] = [0 for sigma_ in range(len(beta_lst))]

load_params["U_bar_00_lst"] = [beta_ - 1 for beta_ in beta_lst]
load_params["U_bar_11_lst"] = [1/beta_-1 for beta_ in beta_lst]

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":len(beta_lst), "Deltat":1., "dt_ini":0.5, "dt_min":0.5, "dt_max":0.5},
    res_basename=res_basename,
    verbose=1
)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
p_hydro_micro_c = qois_vals[:, qois_name_list.index("p_hydro") - 1]
beta_lst_mico_c = [eps_+1 for eps_ in eps_lst]

### Decoupled model

In [ ]:
mat_params = {"model":"CGNH_bar", "parameters":{"E":Es, "nu":nus}}

beta_lst = [x/10 for x in range(10, 21)]
load_params["pf_lst"] = [0 for pf_ in range(len(beta_lst))]
load_params["sigma_bar_01_lst"] = [0 for sigma_ in range(len(beta_lst))]
load_params["sigma_bar_10_lst"] = [0 for sigma_ in range(len(beta_lst))]

load_params["U_bar_00_lst"] = [beta_ - 1 for beta_ in beta_lst]
load_params["U_bar_11_lst"] = [1/beta_-1 for beta_ in beta_lst]

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params=mat_params,
    load_params=load_params,
    step_params={"n_steps":len(beta_lst), "Deltat":1., "dt_ini":0.5, "dt_min":0.5, "dt_max":0.5},
    res_basename=res_basename,
    verbose=1)

qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
p_hydro_micro_d = qois_vals[:, qois_name_list.index("p_hydro") - 1]
beta_lst_mico_d = [eps_+1 for eps_ in eps_lst]

In [ ]:
plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$\beta$', fontsize=16)
plt.ylabel(r'$p_{hydro}~(kPa)$', fontsize=16)

plt.plot(beta_lst_mico_c, p_hydro_micro_c, '#084594')
plt.plot(beta_lst_mico_d, p_hydro_micro_d, '#084594', linestyle='dashed')

plt.plot(beta_vals, p_lst_macro_c, '#99000D')
plt.plot(beta_vals, p_lst_macro_d, '#99000D', linestyle='dashed')

plt.legend(['Microscopic model ' r'$\tilde{p}_{hydro}$'' - coupled', 'Microscopic model ' r'$\tilde{p}_{hydro}$'' - decoupled', 'Macroscopic model ' r'$\bar{p}_{hydro}$'' - coupled', 'Macroscopic model ' r'$\bar{p}_{hydro}$'' - decoupled'])
plt.xlim(1, 2)
plt.ylim(-0.1, 0.01)

plt.show()
# plt.savefig('p_hydro.pdf',bbox_inches='tight')